Recuerda que debes instalar `pandas` antes de poder ejecutar este Notebook. Mi recomendación, si no tienes experiencia en ello, es que descargues [Anaconda](https://www.anaconda.com/). Con esta distribución de Python tendrás acceso a muchas herramientas de desarrollo científico y de análisis de datos en una forma amigable.

Si no deseas utilizar Anaconda, siempre puedes instalar pandas utilizando `pip install pandas`.

In [1]:
import pandas as pd
import os
import json
from collections import defaultdict

In [2]:
# cargar el contenido de todos los archivos JSON a memoria
all_data = []
for file_name in os.listdir('../output/raw'):
    with open(f'../output/raw/{file_name}', 'r') as f:
        all_data.append(json.load(f))

# crear un archivo unificado que contenga los datos
with open('../output/final/resultados.json', 'w') as f:
    json.dump(all_data, f, ensure_ascii=False)

In [3]:
# Crear datos de elección para presidente
structured_data = []
for mesa in all_data:
    n_rec = {}
    for record in mesa['TE'][0].keys():
        n_rec['NROMESA'] = mesa['NROMESA']
        if record != 'VOTOSTE0':
            n_rec[record] = mesa['TE'][0][record]
        else:
            for votos in mesa['TE'][0]['VOTOSTE0']:
                n_rec[f"voto_{votos['D'].replace(' ', '_')}"] = int(votos['V'])
    structured_data.append(n_rec)

# convertir la lista en un dataframe de pandas
main_df = pd.DataFrame(structured_data)
main_df.to_csv('../output/final/resultados.csv', index=None)

In [4]:
# Diputados listado nacional
structured_data = []
for mesa in all_data:
    n_rec = {}
    try:
        for record in mesa['TE'][1].keys():
            if record != 'VOTOSTE1':
                n_rec[record] = mesa['TE'][1][record]
            else:
                for votos in mesa['TE'][1]['VOTOSTE1']:
                    n_rec[f"voto_{votos['D'].replace(' ', '_')}"] = int(votos['V'])
        structured_data.append(n_rec)
    except IndexError:
        pass

# convertir los datos a un dataframe de pandas y exportarlos
dn_df = pd.DataFrame(structured_data)
dn_df.to_csv('../output/final/resultados_dn.csv', index=None)

In [5]:
# diputados distritales
diputados_dict = defaultdict(list)
for mesa in all_data:
    n_rec = {}
    try:
        for record in mesa['TE'][2].keys():
            n_rec['NROMESA'] = mesa['NROMESA']
            if record != 'VOTOSTE2':
                n_rec[record] = mesa['TE'][2][record]
            else:
                for votos in mesa['TE'][2]['VOTOSTE2']:
                    n_rec[f"voto_{votos['D'].replace(' ', '_')}"] = int(votos['V'])
        diputados_dict[mesa["TE"][2]["DEP"]].append(n_rec)
    except IndexError:
        # validando que las mesas de guatemaltecos en USA eran diferentes
        pass
    except KeyError:
        pass

# exportar cada registro a un archivo individual
for departamento in diputados_dict.keys():
    pd.DataFrame(diputados_dict[departamento]).to_csv(f'../output/final/diputados/{departamento}.csv', index=None)

In [6]:
# municipios
muni_dict = defaultdict(list)
for index, mesa in enumerate(all_data):
    n_rec = {}
    try:
        for record in mesa['TE'][3].keys():
            n_rec['NROMESA'] = mesa['NROMESA']
            if record != 'VOTOSTE3':
                n_rec[record] = mesa['TE'][3][record]
            else:
                for votos in mesa['TE'][3]['VOTOSTE3']:
                    n_rec[f"voto_{votos['D'].replace(' ', '_')}"] = int(votos['V'])
        muni_dict[f'{mesa["TE"][3]["DEP"]}-{mesa["TE"][3]["MUN"]}'].append(n_rec)
    except IndexError:
        # validando que las mesas de guatemaltecos en USA eran diferentes
        pass
    except KeyError:
        pass

# exportar cada registro a un archivo individual
for municipio in muni_dict.keys():
    pd.DataFrame(muni_dict[municipio]).to_csv(f'../output/final/municipios/{municipio}.csv', index=None)